In [2]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow #if not using colab delete this line
cap = cv2.VideoCapture('/challenge.mp4') #enter the url of the video
while(cap.isOpened()):
  ret, frame = cap.read() #the frame is like the image
  ###################code edge detection ################################
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

########################end of code##############################
  cv2_imshow(gray) # if not using collab make the function : cv2.imshaow("title",gray)
  if cv2.waitKey(1) & 0xFF == ord('q'):
    break

cap.release()
cv2.destroyAllWindows()
